In [77]:
import numpy as np
import pandas as pd
import torch
from math import factorial
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [16]:
def pos(i,t,prod):
    '''
    Compute positive function and gradient information
    
    input:
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fpt - positive function value
        gfpt - positive function gradient
    '''
    fpt = 0.0 
    gfpt = 0.0 
    fpt = (L/2+prod)**i 
    gfpt = i*(L/2+prod)**(i-1) # no xt yet!
    return fpt,gfpt               

In [4]:
def comb(n, k):
    '''
    Compute combination
    
    input:
        n - total number
        k - number of chosen
    
    output:
        c - number of combination
    '''
    return factorial(n) / factorial(k) / factorial(n - k)

In [5]:
def neg(loss,i,t,prod):
    '''
    Compute negative function and gradient information
    
    input:
        loss - loss function
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fnt - negative function value
        gfnt - negative function gradient
    '''
    fnt = 0.0 # n stands for negative
    gfnt = 0.0
    for k in range(i,N+1):
        # compute forward difference
        delta = 0.0
        for j in range(k+1):
            delta += (-1)**(k-j)*comb(k,j)*loss(i/N)
        # compute coefficient
        beta = comb(N,k)*comb(k,i)*(N+1)*delta/(2*L)**k
        # compute function value
        fnt += beta*(L/2-prod)**(k-i)
        # compute gradient
        gfnt += beta*(k-i)*(L/2-prod)**(k-i-1)  # no xt yet!
    return fnt,gfnt

In [6]:
def p_hat(t,yt,ptm1):
    '''
    Approximate probability
    
    input:
        t - iteration
        yt - label at t
        ptm1 - p at t-1
    
    output:
        pt - p at t
    '''
    pt = ((t-1)*ptm1 + (yt+1)/2)/t # m stands for minus
    return pt

In [7]:
def a_hat(t,fpt,yt,ptm1,atm1):
    '''
    Approximate primal a
    
    input:
        t - iteration
        fpt - positive function at t
        yt - sample label at t
        ptm1 - p at t-1
        atm1 - a at t-1
    
    output:
        at - a at t
    '''
    at = (fpt*((yt+1)/2) + (t-1)*ptm1*atm1)/t # do not update pt yet!
    return at

In [8]:
def b_hat(t,fmt,yt,ptm1,btm1):
    '''
    Approximate primal b
    
    input:
        t - iteration
        fmt - negative function at t
        yt - sample label at t
        ptm1 - p at t-1
        btm1 - b at t-1
    
    output:
        bt - b at t-1
    '''
    bt = (fmt*((-yt+1)/2) + (t-1)*(1-ptm1)*btm1)/t # indicator of y=-1!
    return bt

In [9]:
def alpha_step(t,at,bt):
    '''
    Compute dual alpha
    
    input:
        t - iteration
        at - a at t
        bt - b at t
        
    output:
        alphat - alpha at t
    '''
    alphat = at + bt
    return alphat

In [10]:
def w_grad(fpt,gfpt,fnt,gfnt,yt,pt,at,bt,alphat):
    '''
    Gradient with respect to w
    
    input:
        fpt - positive function at t
        gfpt - positive function gradient at t
        fnt - negative function at t
        gfnt - negative function gradient at t
        yt - sample label at t
        pt - p at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        gradwt - gradient w.r.t. w at t
    '''
    gradwt = 0.0
    if yt == 1:
        gradwt = 2*alphat*(1-pt)*gfpt + 2*(1-pt)*(fpt-at)*gfpt - 2*(1-pt)*fpt*gfpt
    else:
        gradwt = 2*alphat*pt*gfnt + 2*pt*(fnt-bt)*gfnt - 2*pt*fnt*gfnt
    return gradwt

In [11]:
def a_grad(fpt,yt,pt,at):
    '''
    Gradient with respect to a
    
    input:
        fpt - positive function at t
        yt - sample label at t
        pt - p at t
        at - a at t
    
    output:
        gradat - gradient w.r.t a at t
    '''
    gradat = 0.0 
    if yt == 1:
        gradat = -2*(1-pt)*(fpt-at)
    else:
        pass
    return gradat

In [12]:
def b_grad(fnt,yt,pt,bt):
    '''
    Gradient with respect to b
    
    input:
        fnt - negative function at t
        yt - sample label at t
        pt - p at t
        bt - b at t
    
    output:
        gradbt - gradient w.r.t b at t
    '''
    gradbt = 0.0 
    if yt == 1:
        pass
    else:
        gradbt = -2*pt*(fnt-bt)
    return gradbt

In [74]:
def SOLAM(T,loss,pt = 0.0,wt = np.zeros(d), at = 0.0,bt = 0.0,alphat = 0.0):
    '''
    Stochastic Online AUC Maximization
    
    input:
        T - total number of iteration
        loss - loss function
        pt - p at t
        wt - w at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        W - record of each wt
        A - record of each at
        B - record of each bt
        ALPHA - record of each alphat
    '''
    # Initialize W
    W = np.zeros([T,d])
    A = np.zeros(T)
    B = np.zeros(T)
    ALPHA = np.zeros(T)
    
    # Loop
    for t in range(0,T):
        
        # Update pt
        pt = p_hat(t,y[t],pt)
        
        # Update wt,at,bt
        prod = np.inner(wt,x[t])
        fpt = np.zeros(N+1)
        gfpt = np.zeros(N+1)
        fnt = np.zeros(N+1)
        gfnt = np.zeros(N+1)
        gradwt = np.zeros(d)
        gradat = 0.0
        gradbt = 0.0
        
        for i in range(N+1): # add up info of each i
            fpt[i],gfpt[i] = pos(i,t,prod) # partial info
            fnt[i],gfnt[i] = neg(loss,i,t,prod)
            gradwt += w_grad(fpt[i],gfpt[i],fnt[i],gfnt[i],y[t],pt,at,bt,alphat)
            gradat += a_grad(fpt[i],y[t],pt,at)
            gradbt += b_grad(fnt[i],y[t],pt,bt)
            
        wt -= 1/(t+1)*gradwt*y[t]*x[t]/(N+1) # step size as 1/t gradient descent
        at -= 1/(t+1)*gradat/(N+1)
        bt -= 1/(t+1)*gradbt/(N+1)
        alphat = at+bt
        
        W[t,:] = wt
        A[t] = at
        B[t] = bt
        ALPHA[t] = alphat
        
        print('iteration: %d' % (t+1))
    return W,A,B,ALPHA

In [68]:
def loader(filename):
    '''
    Data file loader
    
    input:
        filename - filename
    
    output:
        x - sample features
        y - sample labels
    '''
    raw_df = pd.read_csv(filename,header=None,sep = '\s+|:',engine='python')
    y = raw_df[0].values
    x = raw_df[raw_df.columns[2::2]].values
    return x,y

In [69]:
L = 1.0 # range
N = 10 # degree

In [70]:
x,y = loader('diabetes')
T,d = x.shape

In [75]:
hinge = lambda x:max(0,x)
epochs = 1
W,A,B,ALPHA = SOLAM(T,hinge)
for epoch in range(epochs):
    pass

/home/neyo/PycharmProjects/AUC/venv/lib/python3.5/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29
iteration: 30
iteration: 31
iteration: 32
iteration: 33
iteration: 34
iteration: 35
iteration: 36
iteration: 37
iteration: 38
iteration: 39
iteration: 40
iteration: 41
iteration: 42
iteration: 43
iteration: 44
iteration: 45
iteration: 46
iteration: 47
iteration: 48
iteration: 49
iteration: 50
iteration: 51
iteration: 52
iteration: 53
iteration: 54
iteration: 55
iteration: 56
iteration: 57
iteration: 58
iteration: 59
iteration: 60
iteration: 61
iteration: 62
iteration: 63
iteration: 64
iteration: 65
iteration: 66
iteration: 67
iteration: 68
iteration: 69
iteration: 70
iteration: 71
iteration: 72
i

In [84]:
fpr, tpr, _ = roc_curve(y, np.dot(x,W[T-1]))
roc_auc = auc(fpr, tpr)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').